# Solving linear equations

# I. $LU$ factorization of a square matrix

Let us consider a decomposition of a square $n \times n$ matrix A as follows:
$$A = L \cdot U, \; \mbox{where} \; A = \begin{pmatrix} 
                                            a_{11} & a_{12} & a_{13} & \ldots & a_{1n} \\
                                            a_{21} & a_{22} & a_{23} & \ldots & a_{2n} \\
                                            a_{31} & a_{32} & a_{33} & \ldots & a_{3n} \\
                                            \vdots & \vdots & \vdots & \ddots & \vdots \\
                                            a_{n1} & a_{n2} & a_{n3} & \ldots & a_{nn} \\
                                        \end{pmatrix}
                               , \; L = \begin{pmatrix} 
                                            1 & 0 & 0 & \ldots & 0 \\
                                            * & 1 & 0 & \ldots & 0 \\
                                            * & * & 1 & \ldots & 0 \\
                                            \vdots & \vdots & \vdots & \ddots & \vdots \\
                                            * & * & * & \ldots & 1 \\
                                        \end{pmatrix}
                               , \; U = \begin{pmatrix} 
                                            a_{11} & * & * & \ldots & * \\
                                            0 & a_{22} & * & \ldots & * \\
                                            0 & 0 & a_{33} & \ldots & * \\
                                            \vdots & \vdots & \vdots & \ddots & \vdots \\
                                            0 & 0 & 0 & \ldots & a_{nn} \\
                                        \end{pmatrix} .$$

Let's start with Gaussian elimination. When we are working with the first column, we combine the first row and the second row multiplied by coefficient $$\gamma_{21} = \cfrac{a_{21}}{a{11}};$$ then the first row and the third row multiplied by coefficient $$\gamma_{31} = \cfrac{a_{31}}{a{11}},$$ and so on.

Hereby, to eliminate all elements below $a_{11}$ we need to multiply matrix A by matrix $$\Lambda_1 = \begin{pmatrix} 
                                                        1 & 0 & 0 & \ldots & 0 \\
                                                        -\gamma_{21} & 1 & 0 & \ldots & 0 \\
                                                        -\gamma_{31} & 0 & 1 & \ldots & 0 \\
                                                        \vdots & \vdots & \vdots & \ddots & \vdots \\
                                                        -\gamma_{n1} & 0 & 0 & \ldots & 1 \\
                                                    \end{pmatrix}$$
(prove it by trying to act with $\Lambda_1$ on the first column of matrix A).

Likewise, we can construct matrix $\Lambda_2$ as $$\Lambda_2 = \begin{pmatrix} 
                                                                1 & 0 & 0 & \ldots & 0 \\
                                                                0 & 1 & 0 & \ldots & 0 \\
                                                                0 & -\gamma_{32} & 1 & \ldots & 0 \\
                                                                \vdots & \vdots & \vdots & \ddots & \vdots \\
                                                                0 & -\gamma_{n2} & 0 & \ldots & 1 \\
                                                            \end{pmatrix}.$$

Finally, we will get the upper triangular matrix $$U = \Lambda_n \cdot \Lambda_{n-1} \cdot \ldots \cdot \Lambda_2 \cdot \Lambda_1 A. $$
Hence the lower triangular matrix $L = \Lambda_1^{-1} \cdot \Lambda_2^{-1} \cdot \ldots \cdot \Lambda_{n-1}^{-1} \cdot \Lambda_n^{-1}.$

One can show that, for example, $$\Lambda_1^{-1} = \begin{pmatrix} 
                                                        1 & 0 & 0 & \ldots & 0 \\
                                                        \gamma_{21} & 1 & 0 & \ldots & 0 \\
                                                        \gamma_{31} & 0 & 1 & \ldots & 0 \\
                                                        \vdots & \vdots & \vdots & \ddots & \vdots \\
                                                        \gamma_{n1} & 0 & 0 & \ldots & 1 \\
                                                    \end{pmatrix}$$

Note that we're using the `numpy` arrays to represent matrices [do **not** use `np.matrix`].

In [35]:
import numpy as np

def diy_lu_ext(a):
    """
    Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    #Initializing the factors
    u = a.copy()
    L = np.eye(N)
    
    for j in range(N-1):
        lam = np.eye(N)
        
        #Creating the vector of gammas
        gamma = np.zeros(N-j-1)
        for i in range(N-j-1):
            gamma[i] = u[j+1+i, j]/u[j, j]
        
        #Creating matrix \Lambda_i
        for i in range(N-j-1):
            lam[j+1+i, j] = -gamma[i]
        
        #Acting with \Lambda_i on A to get U
        u_new = np.zeros((N, N))
        for ind_i in range(N):
            for ind_j in range(N):
                for ind_k in range(N):
                    u_new[ind_i, ind_j] += lam[ind_i, ind_k] * u[ind_k, ind_j]
        u = u_new.copy()
        
        #Creating matrix \Lambda_i^{-1}
        for i in range(N-j-1):
            lam[j+1+i, j] = gamma[i]
            
        #Multiplying L and \Lambda_i^{-1} o get new L
        L_new = np.zeros((N, N))
        for ind_i in range(N):
            for ind_j in range(N):
                for ind_k in range(N):
                    L_new[ind_i, ind_j] += L[ind_i, ind_k] * lam[ind_k, ind_j]
        L = L_new.copy()
        
    return L, u

In [36]:
# Now, generate a full rank matrix and test the naive implementation

import numpy as np

N = 6
a = np.zeros((N, N), dtype=float)
for i in range(N):
    for j in range(N):
        a[i, j] = 3. / (0.6*i*j + 1)

np.linalg.matrix_rank(a)

6

In [37]:
L, u = diy_lu_ext(a)
print(L@u - a)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -3.886e-16  3.886e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.665e-16 -1.665e-16 -1.665e-16  0.000e+00]]


LU can be programmed in a more simple way by using the perks of `numpy`.

In [38]:
import numpy as np

def diy_lu(a):
    """
    Construct the LU decomposition of the input matrix.
    
    Naive LU decomposition: work column by column, accumulate elementary triangular matrices.
    No pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    for j in range(N-1):
        lam = np.eye(N)
        
        #Creating the vector of gammas
        gamma = u[j+1:, j] / u[j, j]
        
        #Creating matrix \Lambda_i
        lam[j+1:, j] = -gamma
        
        
        #Acting with \Lambda_i on A to get U
        u = lam @ u
        
        
        #Creating matrix \Lambda_i^{-1}
        lam[j+1:, j] = gamma
            
        #Multiplying L and \Lambda_i^{-1} o get new L
        L = L @ lam
    return L, u

In [39]:
# Tweak the printing of floating-point numbers, for clarity
np.set_printoptions(precision=3)

In [40]:
L, u = diy_lu(a)

print(L, "\n")
print(u, "\n")

# Quick sanity check: L times U must equal the original matrix, up to floating-point errors.
print(L@u - a)

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    1.455 1.    0.    0.    0.   ]
 [1.    1.714 1.742 1.    0.    0.   ]
 [1.    1.882 2.276 2.039 1.    0.   ]
 [1.    2.    2.671 2.944 2.354 1.   ]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -1.125e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  2.625e-01  4.574e-01  5.975e-01  7.013e-01]
 [ 0.000e+00  2.220e-16  0.000e+00 -2.197e-02 -4.480e-02 -6.469e-02]
 [ 0.000e+00 -4.528e-16  0.000e+00  6.939e-18  8.080e-04  1.902e-03]
 [ 0.000e+00  4.123e-16  0.000e+00 -1.634e-17  0.000e+00 -1.585e-05]] 

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 0.000e+00  0.000e+00  0.000e+00  2.220e-16 -1.110e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00  2.220e-16 -5.551e-17 -1.665e-16 -1.665e-16]
 [ 0.000e+00  0.000e+00 -1.110e-16 -3.886e-16  3.886e-16 -1.665e-16]
 

# II. The need for pivoting

Let's tweak the matrix a little bit, we only change a single element:

In [41]:
a1 = a.copy()
a1[1, 1] = 3

Resulting matrix still has full rank, but the naive LU routine breaks down.

In [42]:
np.linalg.matrix_rank(a1)

6

In [43]:
l, u = diy_lu(a1)

print(l, u)

[[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]] [[nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]
 [nan nan nan nan nan nan]]


/home/liza/.local/lib/python3.5/site-packages/ipykernel_launcher.py:18: RuntimeWarning: divide by zero encountered in true_divide
/home/liza/.local/lib/python3.5/site-packages/ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in matmul
/home/liza/.local/lib/python3.5/site-packages/ipykernel_launcher.py:32: RuntimeWarning: invalid value encountered in matmul
/home/liza/.local/lib/python3.5/site-packages/ipykernel_launcher.py:18: RuntimeWarning: invalid value encountered in true_divide


### Test II.1

For a naive LU decomposition to work, all leading minors of a matrix should be non-zero. Check if this requirement is satisfied for the two matrices `a` and `a1`.

In [44]:
import sympy
from sympy import Matrix

In [45]:
m = sympy.Matrix(a)
m

Matrix([
[3.0,               3.0,               3.0,               3.0,               3.0,               3.0],
[3.0,             1.875,  1.36363636363636,  1.07142857142857, 0.882352941176471,              0.75],
[3.0,  1.36363636363636, 0.882352941176471, 0.652173913043478, 0.517241379310345, 0.428571428571429],
[3.0,  1.07142857142857, 0.652173913043478,           0.46875, 0.365853658536585,               0.3],
[3.0, 0.882352941176471, 0.517241379310345, 0.365853658536585, 0.283018867924528, 0.230769230769231],
[3.0,              0.75, 0.428571428571429,               0.3, 0.230769230769231,            0.1875]])

In [46]:
# leading minors
def minors(matrix):
    result_minors = []
    for j in range(len(a)):
        for i in range(len(a)):
            if j==i:
                result_minors.append(m.minorEntry(j,i)) 
    return result_minors
            

In [47]:
# for matrix a principal minors
m.berkowitz_minors()

(1,
 3.00000000000000,
 -3.37500000000000,
 -0.885999027710257,
 0.0194670010320019,
 1.57289732825071e-5,
 -2.59116033371563e-10)

In [48]:
# for matrix a1 principal minors
m1 = sympy.Matrix(a1)
m1.berkowitz_minors()

(1,
 3.00000000000000,
 0,
 -8.03305785123966,
 -0.493510106162495,
 0.000903014520269169,
 2.31734814549328e-8)

In [49]:
# for matrix a leading minors
minors(m)

[7.91046850723931e-14,
 2.08293969542529e-8,
 4.25862737204843e-6,
 5.29684242987418e-5,
 8.68863901253511e-5,
 1.57289741545937e-5]

In [50]:
m1 = sympy.Matrix(a1)
m1

Matrix([
[3.0,               3.0,               3.0,               3.0,               3.0,               3.0],
[3.0,               3.0,  1.36363636363636,  1.07142857142857, 0.882352941176471,              0.75],
[3.0,  1.36363636363636, 0.882352941176471, 0.652173913043478, 0.517241379310345, 0.428571428571429],
[3.0,  1.07142857142857, 0.652173913043478,           0.46875, 0.365853658536585,               0.3],
[3.0, 0.882352941176471, 0.517241379310345, 0.365853658536585, 0.283018867924528, 0.230769230769231],
[3.0,              0.75, 0.428571428571429,               0.3, 0.230769230769231,            0.1875]])

In [51]:
# for matrix a1 leading minors
minors(m1)

[7.91046850723931e-14,
 2.08293969542529e-8,
 4.25862737204843e-6,
 5.29684242987418e-5,
 8.68863901253511e-5,
 1.57289741545937e-5]

###### Вывод: один из угловых миноров для матрицы а1 равен 0, что не позволяет применить LU разложение для этой матрицы.

### Test II.2

Modify the `diy_lu` routine to implement column pivoting. Keep track of pivots, you can either construct a permutation matrix, or a swap array (your choice).

Implement a function to reconstruct the original matrix from a decompositon. Test your routines on the matrices `a` and `a1`.

#### Для матрицы а

In [52]:
def diy_lu_pivot(a):
    """
    Construct the LUP decomposition of the input matrix.
    
    With column pivoting.
    """
    N = a.shape[0]
    
    u = a.copy()
    L = np.eye(N)
    piv = np.eye(N)
    P = np.eye(N)
    for j in range(N-1):
        # Pivoting         
        max_column_index = np.argmax(abs(u[j:,j]))+j
        if max_column_index != j:
            u[[max_column_index, j], j:N] = u[[j, max_column_index], j:N]
            piv[[max_column_index, j], j:N] = piv[[j, max_column_index], j:N]
            P = piv@P
            L = piv@L
        # Find U   
        lam = np.eye(N)
        for k in range(j+1, N):
            gamma = (u[k, j] / u[j, j])
            lam[k, j]  = -gamma
            L[k, j] =  gamma       
            u = lam @ u          
    np.fill_diagonal(L, 1)   
    return P, L, u



In [53]:
P, L, U = diy_lu_pivot(a)
print(P, '\n', L, '\n', U)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]] 
 [[ 1.     0.     0.     0.     0.     0.   ]
 [ 1.     1.     0.     0.     0.     0.   ]
 [ 1.     0.81   1.     0.     0.     0.   ]
 [ 1.     0.604  0.495  1.     0.     0.   ]
 [ 1.     0.39   0.156  0.158  1.     0.   ]
 [ 1.     0.171 -0.007  0.002 -0.006  1.   ]] 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [-1.200e+01 -1.312e+01 -1.364e+01 -1.393e+01 -1.412e+01 -1.425e+01]
 [ 2.990e+01  3.191e+01  3.309e+01  3.380e+01  3.428e+01  3.462e+01]
 [-2.862e+01 -3.149e+01 -3.273e+01 -3.345e+01 -3.392e+01 -3.425e+01]
 [ 6.114e+00  5.158e+00  5.217e+00  5.298e+00  5.363e+00  5.412e+00]
 [ 2.368e+00  3.257e-01  1.034e-01  3.202e-02  0.000e+00 -1.715e-02]]


In [54]:
print(P@a-L@U)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.200e+01  1.200e+01  1.200e+01  1.200e+01  1.200e+01  1.200e+01]
 [-2.017e+01 -2.291e+01 -2.415e+01 -2.486e+01 -2.532e+01 -2.564e+01]
 [ 2.108e+01  2.171e+01  2.225e+01  2.261e+01  2.285e+01  2.303e+01]
 [-1.557e+00 -2.138e+00 -2.350e+00 -2.466e+00 -2.540e+00 -2.591e+00]
 [ 0.000e+00  2.220e-16  1.665e-16 -5.551e-17  5.551e-17  0.000e+00]]


#### Функция для получения исходной матрицы а: $PA=LU$ отсюда $A=P^{-1}LU$

In [55]:
P_inv = np.linalg.inv(P)

In [56]:
print(a - P_inv @ L @ U)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.200e+01  1.200e+01  1.200e+01  1.200e+01  1.200e+01  1.200e+01]
 [-2.017e+01 -2.291e+01 -2.415e+01 -2.486e+01 -2.532e+01 -2.564e+01]
 [ 2.108e+01  2.171e+01  2.225e+01  2.261e+01  2.285e+01  2.303e+01]
 [-1.557e+00 -2.138e+00 -2.350e+00 -2.466e+00 -2.540e+00 -2.591e+00]
 [ 0.000e+00  2.220e-16  1.665e-16 -5.551e-17  5.551e-17  0.000e+00]]


#### Для матрицы а1

In [57]:
P1, L1, U1 = diy_lu_pivot(a1)
print(P1, '\n', L1, '\n', U1)

[[1. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]] 
 [[ 1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  8.864e-01  1.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  6.607e-01  5.018e-01  1.000e+00  0.000e+00  0.000e+00]
 [ 1.000e+00  4.265e-01  1.651e-01  1.654e-01  1.000e+00  0.000e+00]
 [ 1.000e+00  1.875e-01 -3.924e-04  1.924e-03 -5.089e-03  1.000e+00]] 
 [[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [-1.200e+01 -1.200e+01 -1.364e+01 -1.393e+01 -1.412e+01 -1.425e+01]
 [ 3.355e+01  3.191e+01  3.723e+01  3.804e+01  3.857e+01  3.895e+01]
 [-3.271e+01 -3.218e+01 -3.736e+01 -3.818e+01 -3.871e+01 -3.909e+01]
 [ 6.980e+00  5.226e+00  6.214e+00  6.317e+00  6.395e+00  6.454e+00]
 [ 2.362e+00  1.010e-01  1.035e-01  3.214e-02  0.000e+00 -1.728e-02]]


#### Функция для получения исходной матрицы а: $PA=LU$ отсюда $A=P^{-1}LU$

In [58]:
P1_inv = np.linalg.inv(P1)

In [59]:
print(a - P1_inv @ L1 @ U1)

[[ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00]
 [ 1.200e+01  1.088e+01  1.200e+01  1.200e+01  1.200e+01  1.200e+01]
 [-2.291e+01 -2.291e+01 -2.726e+01 -2.804e+01 -2.854e+01 -2.889e+01]
 [ 2.381e+01  2.217e+01  2.534e+01  2.577e+01  2.605e+01  2.626e+01]
 [-1.990e+00 -2.172e+00 -2.849e+00 -2.975e+00 -3.056e+00 -3.112e+00]
 [ 0.000e+00 -1.110e-16  1.665e-16  2.776e-16 -2.776e-17  0.000e+00]]


### Scipy

In [60]:
import scipy
from scipy import linalg
P_sp, L_sp, U_sp = scipy.linalg.lu(a)
print(P_sp, "\n")
print(U_sp, "\n")
print(L_sp, "\n")

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -3.506e-01 -5.786e-01 -7.330e-01 -8.438e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  2.421e-02  4.866e-02  6.961e-02]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00 -6.462e-04 -1.516e-03]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  6.730e-06]] 

[[1.    0.    0.    0.    0.    0.   ]
 [1.    1.    0.    0.    0.    0.   ]
 [1.    0.5   1.    0.    0.    0.   ]
 [1.    0.727 0.706 1.    0.    0.   ]
 [1.    0.857 0.41  0.835 1.    0.   ]
 [1.    0.941 0.178 0.426 0.789 1.   ]] 



In [61]:
P1_sp, L1_sp, U1_sp = scipy.linalg.lu(a1)
print(P1_sp, "\n")
print(U1_sp, "\n")
print(L1_sp, "\n")

[[1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 1.]
 [0. 1. 0. 0. 0. 0.]] 

[[ 3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00  3.000e+00]
 [ 0.000e+00 -2.250e+00 -2.571e+00 -2.700e+00 -2.769e+00 -2.812e+00]
 [ 0.000e+00  0.000e+00 -1.636e+00 -1.929e+00 -2.118e+00 -2.250e+00]
 [ 0.000e+00  0.000e+00  0.000e+00 -9.247e-02 -1.485e-01 -1.856e-01]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  1.841e-03  3.821e-03]
 [ 0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00 -1.233e-05]] 

[[ 1.     0.     0.     0.     0.     0.   ]
 [ 1.     1.     0.     0.     0.     0.   ]
 [ 1.    -0.     1.     0.     0.     0.   ]
 [ 1.     0.727  0.151  1.     0.     0.   ]
 [ 1.     0.857  0.088  0.514  1.     0.   ]
 [ 1.     0.941  0.038  0.208  0.641  1.   ]] 



Sum all elements in matrix `L1` and `u1` separately (for Google Form).

In [62]:
# for L1
sum_L1 = 0
for i in range(len(L1)):
    for j in range(len(L1)):
        sum_L1 += L1[i][j]
print(sum_L1)
        

13.989868619883875


In [63]:
# for u1
sum_U1 = 0
for i in range(len(U1)):
    for j in range(len(U1)):
        sum_U1 += U1[i][j]
print(sum_U1)

-21.763141744694217


In [64]:
# for L1_sp
sum_L1 = 0
for i in range(len(L1_sp)):
    for j in range(len(L1_sp)):
        sum_L1 += L1_sp[i][j]
print(sum_L1)

15.166249240140331


In [65]:
# for U1_sp
sum_U1 = 0
for i in range(len(U1_sp)):
    for j in range(len(U1_sp)):
        sum_U1 += U1_sp[i][j]
print(sum_U1)

-3.456659008328081


###### Вывод: в моем алгоритме где-то ошибка, но найти ее я не смогла((